In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

We will use keras tuner to hyper tune the models

In [2]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 7.8 MB/s 


In [3]:
import keras_tuner as kt
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

Copying the dataset

In [4]:
df = pd.read_csv('data/simpleData.csv')

In [5]:
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [6]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('price')
test_labels = test_features.pop('price')

In [7]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

Defining a model

In [12]:
def build_and_compile_model(hp):
  model = keras.Sequential([
      normalizer,
      layers.Dense(units=hp.Int("units_l1", 16, 512, step=16),
                          use_bias=hp.Boolean("bias_l1"),
                          activation=hp.Choice("act_l1", ["relu", "tanh"])),
      layers.Dense(units=hp.Int("units_l2", 16, 512, step=16), 
                          use_bias=hp.Boolean("bias_l2"),
                          activation=hp.Choice("act_l2", ["relu", "tanh"])),
      layers.Dense(units=hp.Int("units_l3", 16, 512, step=16),
                          use_bias=hp.Boolean("bias_l3"),
                          activation=hp.Choice("act_l3", ["relu", "tanh"])),
      layers.Dense(1)
  ])

  optim=hp.Choice("optimizer",["sgd","rmsprop","adam"])
  
  model.compile(optimizer=optim,
                loss="mean_squared_error", metrics=["mean_squared_error"])
  return model

We are using a hyperband tuner and also adding a call back tp stop training after a certain validation loss

In [13]:
tuner = kt.Hyperband(build_and_compile_model,
                    objective=kt.Objective(name="val_loss",direction="min"),
                     max_epochs=10,
                     #max_trials=3,
                     #executions_per_trial=2,
                     factor=3,
                     directory='models',
                     project_name='ride_model_param_tuning')

INFO:tensorflow:Reloading Oracle from existing project models/ride_model_param_tuning/oracle.json


In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', patience=5)

In [15]:
tuner.search(train_features, train_labels, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 01m 23s]
val_loss: 7.643059730529785

Best val_loss So Far: 7.542886257171631
Total elapsed time: 00h 15m 19s
INFO:tensorflow:Oracle triggered exit


In [17]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

print(f"""
The hyperparameter search is complete. The optimizer of units in the densely-connected
layer1 {best_hps.get('units_l1')}, activation {best_hps.get('act_l1')} and bias {best_hps.get('bias_l1')}
layer2 {best_hps.get('units_l2')}, activation {best_hps.get('act_l2')} and bias {best_hps.get('bias_l2')}
layer3 {best_hps.get('units_l3')}, activation {best_hps.get('act_l3')} and bias {best_hps.get('bias_l3')}
is {best_hps.get('optimizer')}.
""")


The hyperparameter search is complete. The optimizer of units in the densely-connected
layer1 480, activation tanh and bias True
layer2 112, activation tanh and bias False
layer3 224, activation tanh and bias True
is adam.



We can use the results from tuning the parameters to build the model

In [19]:
def build_model(normalizer):
  model = keras.Sequential([
      normalizer,
      layers.Dense(units=480,
                          use_bias=True,
                          activation="tanh"),
      layers.Dense(units=112,
                          use_bias=False,
                          activation="tanh"),
      layers.Dense(units=224,
                          use_bias=True,
                          activation="tanh"),
      layers.Dense(1)
  ])  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [20]:
dnn_model = build_model(normalizer)

In [21]:
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 5)                11        
 n)                                                              
                                                                 
 dense_4 (Dense)             (None, 480)               2880      
                                                                 
 dense_5 (Dense)             (None, 112)               53760     
                                                                 
 dense_6 (Dense)             (None, 224)               25312     
                                                                 
 dense_7 (Dense)             (None, 1)                 225       
                                                                 
Total params: 82,188
Trainable params: 82,177
Non-trainable params: 11
_________________________________________________

In [22]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=2, epochs=100)

Epoch 1/100
1925/1925 - 6s - loss: 4.5078 - val_loss: 1.7178 - 6s/epoch - 3ms/step
Epoch 2/100
1925/1925 - 5s - loss: 1.5183 - val_loss: 1.4863 - 5s/epoch - 2ms/step
Epoch 3/100
1925/1925 - 5s - loss: 1.4398 - val_loss: 1.4712 - 5s/epoch - 2ms/step
Epoch 4/100
1925/1925 - 5s - loss: 1.4126 - val_loss: 1.4028 - 5s/epoch - 2ms/step
Epoch 5/100
1925/1925 - 5s - loss: 1.3992 - val_loss: 1.4249 - 5s/epoch - 2ms/step
Epoch 6/100
1925/1925 - 5s - loss: 1.4023 - val_loss: 1.3807 - 5s/epoch - 2ms/step
Epoch 7/100
1925/1925 - 5s - loss: 1.3707 - val_loss: 1.4237 - 5s/epoch - 2ms/step
Epoch 8/100
1925/1925 - 5s - loss: 1.3728 - val_loss: 1.3887 - 5s/epoch - 2ms/step
Epoch 9/100
1925/1925 - 5s - loss: 1.3600 - val_loss: 1.3931 - 5s/epoch - 2ms/step
Epoch 10/100
1925/1925 - 5s - loss: 1.3518 - val_loss: 1.3975 - 5s/epoch - 2ms/step
Epoch 11/100
1925/1925 - 5s - loss: 1.3484 - val_loss: 1.3558 - 5s/epoch - 2ms/step
Epoch 12/100
1925/1925 - 5s - loss: 1.3566 - val_loss: 1.3530 - 5s/epoch - 3ms/step
E

In [23]:
dnn_model.save('models/trained_model')

INFO:tensorflow:Assets written to: models/trained_model/assets


INFO:tensorflow:Assets written to: models/trained_model/assets


In [27]:
!zip -r models.zip models

  adding: models/ (stored 0%)
  adding: models/ride_model_param_tuning/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_0017/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_0017/trial.json (deflated 77%)
  adding: models/ride_model_param_tuning/trial_0017/checkpoint.data-00000-of-00001 (deflated 8%)
  adding: models/ride_model_param_tuning/trial_0017/checkpoint.index (deflated 52%)
  adding: models/ride_model_param_tuning/trial_0017/checkpoint (deflated 48%)
  adding: models/ride_model_param_tuning/trial_0012/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_0012/trial.json (deflated 77%)
  adding: models/ride_model_param_tuning/trial_0012/checkpoint.data-00000-of-00001 (deflated 8%)
  adding: models/ride_model_param_tuning/trial_0012/checkpoint.index (deflated 52%)
  adding: models/ride_model_param_tuning/trial_0012/checkpoint (deflated 48%)
  adding: models/ride_model_param_tuning/trial_0025/ (stored 0%)
  adding: models/ride_model_param_tuning/trial_